# Vapor Pressure Curves

Comparing vapor pressure correlations for water by sweeping temperature through Antoine, Kirchhoff, and Wagner blocks wired into a PathSim simulation.

The IK-CAPE standard defines several temperature-dependent correlations for vapor pressure. Here we compare three of them:

- **Antoine**: $\ln P = a_0 - a_1 / (T + a_2)$
- **Kirchhoff**: $\ln P = a_0 - a_1/T - a_2 \ln T$
- **Wagner**: $\ln P = \ln P_c + (1/T_r)(a_2\tau + a_3\tau^{1.5} + a_4\tau^3 + a_5\tau^6)$

Each correlation is a `Function` block with a single temperature input and a single pressure output.

In [ ]:
import matplotlib.pyplot as plt

from pathsim import Simulation, Connection
from pathsim.blocks import Source, Scope

from pathsim_chem.thermodynamics import Antoine, Kirchhoff, Wagner

Define the correlation blocks with literature coefficients for water. All use natural logarithm, temperature in Kelvin, and return pressure in Pascals.

In [ ]:
# Antoine coefficients for water (NIST)
antoine = Antoine(a0=23.2256, a1=3835.18, a2=-45.343)

# Kirchhoff coefficients for water
kirchhoff = Kirchhoff(a0=50.4338, a1=6350.4068, a2=3.6963)

# Wagner coefficients for water (Tc=647.096 K, Pc=22.064 MPa)
wagner = Wagner(Tc=647.096, Pc=22064000, a2=-7.76451, a3=1.45838, a4=-2.77580, a5=-1.23303)

Set up a simulation that sweeps temperature from 300 K to 500 K. A `Source` block generates a linear temperature ramp, which is connected to all three correlations in parallel. A `Scope` records the three pressure outputs.

In [ ]:
# Temperature ramp: 300 K to 500 K over 200 seconds
T_src = Source(func=lambda t: 300 + t)

# Scope to record all three vapor pressures
scp = Scope(labels=["Antoine", "Kirchhoff", "Wagner"])

sim = Simulation(
    blocks=[T_src, antoine, kirchhoff, wagner, scp],
    connections=[
        Connection(T_src, antoine, kirchhoff, wagner),
        Connection(antoine, scp),
        Connection(kirchhoff, scp[1]),
        Connection(wagner, scp[2]),
    ],
    dt=1.0,
)

sim.run(200)

In [ ]:
time, signals = scp.read()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

T_celsius = time + 300 - 273.15
labels = ["Antoine", "Kirchhoff", "Wagner"]
styles = ["-", "--", ":"]

for i, (label, style) in enumerate(zip(labels, styles)):
    ax1.plot(T_celsius, signals[i] / 1e3, style, label=label)
    ax2.semilogy(1000 / (time + 300), signals[i], style, label=label)

ax1.axhline(101.325, color="gray", linestyle="-.", alpha=0.5, label="1 atm")
ax1.set_xlabel("Temperature [°C]")
ax1.set_ylabel("Vapor Pressure [kPa]")
ax1.set_title("Vapor Pressure of Water")
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.set_xlabel("1000 / T [1/K]")
ax2.set_ylabel("Vapor Pressure [Pa]")
ax2.set_title("Clausius-Clapeyron Plot")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

All three correlations agree well in their common validity range. The Wagner equation is typically preferred for high-accuracy work since it is constrained to reach the critical point.